In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import pandas as pd
import glob as glob
import time
import pickle
from omegaconf import OmegaConf

# add custom paths
sys.path.extend([os.path.abspath('./assetto_corsa_gym')])
import AssettoCorsaEnv.assettoCorsa as assettoCorsa
from AssettoCorsaEnv.brake_map import BrakeMap

# Configure the logging system
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,  # Set the logging level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Format of the log messages
    datefmt='%Y-%m-%d %H:%M:%S',  # Format of the timestamp
)



In [ ]:
# Create environment
config = OmegaConf.load("config.yml")

# Set the configuration
config.AssettoCorsa.use_relative_actions = False
config.AssettoCorsa.enable_low_speed_termination = False
config.AssettoCorsa.car = "dallara_f317" # ks_mazda_miata" # "bmw_z4_gt3"

env = assettoCorsa.make_ac_env(cfg=config, work_dir="output")

# Create the brake map by setting values throughout the full range and observing the telemetry

In [ ]:
res = []

env.reset()
for b in np.linspace(-1,1, 21):
    for i in range(25):
        _, _, done, _ = env.step([0.0, -1.0, b])
    r = {'brake': b, 'brakeStatus': env.state['brakeStatus']}
    res.append(r.copy())
    print(r)

# Save
x = [r['brake'] for r in res]
y = [r['brakeStatus'] for r in res]
brake_map = BrakeMap(x, y, kind='cubic')
brake_map.save()

In [ ]:
#brake_map = BrakeMap.load()

In [ ]:
y_int = brake_map.get_y(np.arange(-1, 1, 0.01))

# Create the plot
plt.figure(figsize=(8, 6))
plt.plot(x, y, 'o-', label='Data Points', alpha=0.5)  # 'o-' creates a line plot with circle markers
plt.scatter(np.arange(-1, 1, 0.01), y_int, s=2, label='Data Points', color='red')  # 'o-' creates a line plot with circle markers

# Adding plot labels and title
plt.xlabel('brake')
plt.ylabel('brake AC Feedback')
plt.title('brake map forward')
plt.grid(True)  # Show grid
plt.legend()  # Show legend

# Display the plot
plt.show()

y = np.arange(0, 1, 0.01) # brake status
x_int = brake_map.get_x(y)

# Create the plot
plt.figure(figsize=(8, 6))
plt.scatter(y, x_int, s=2, label='Data Points', color='red')  # 'o-' creates a line plot with circle markers

# Adding plot labels and title
plt.xlabel('brake AC Feedback')
plt.ylabel('brake')
plt.title('Reverse')
plt.grid(True)  # Show grid
plt.legend()  # Show legend

# Display the plot
plt.show()

# steer map

In [ ]:
res = []

env.reset()
for b in [-1 , 1]:
    for i in range(25):
        _, _, done, _ = env.step([b, -1.0, -1.0])
    r = {'action_steerAngle': b, 'steerAngle': env.state['steerAngle']}
    res.append(r.copy())
print(res)